In [11]:
import numpy as np
import pandas as pd
import warnings
from sklearn.model_selection import (
    train_test_split, GridSearchCV, RandomizedSearchCV,
    KFold, StratifiedKFold, ShuffleSplit, LeaveOneOut, GroupKFold
)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from scipy.stats import uniform

# Ігнорування попереджень
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

# 1. Завантаження даних
df = pd.read_csv("boston.csv")

# 2. Попередня обробка
if 'CHAS' in df.columns:
    le = LabelEncoder()
    df['CHAS'] = le.fit_transform(df['CHAS'])

scaler = StandardScaler()
df[df.columns] = scaler.fit_transform(df)

X = df.drop(columns=['MEDV'])
y = (df['MEDV'] > df['MEDV'].median()).astype(int)

# 3. Розділення
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# 4. Параметри для пошуку
param_grid = {
    'C': np.logspace(-3, 3, 10),
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga']
}
param_dist = {
    'C': uniform(0.01, 10),
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga']
}

# 5. Функція для оцінки
def evaluate_model_with_cv(search_type, X_train, y_train, cv_strategy):
    if search_type == 'grid':
        model = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, cv=cv_strategy, scoring='accuracy')
    else:
        model = RandomizedSearchCV(LogisticRegression(max_iter=1000), param_dist, n_iter=10, cv=cv_strategy, scoring='accuracy', random_state=42)
    
    model.fit(X_train, y_train)
    print(f"{search_type.capitalize()} Search | CV: {type(cv_strategy).__name__}")
    print("Best params:", model.best_params_)
    print("Best score:", model.best_score_)
    print("-" * 60)

# 6. Варіанти крос-валідації
cv_strategies = [
    KFold(n_splits=5, shuffle=True, random_state=42),         # З перемішуванням
    KFold(n_splits=5, shuffle=False),                         # Без перемішування
    StratifiedKFold(n_splits=5),                              # Стратифікована
    ShuffleSplit(n_splits=5, test_size=0.3, random_state=42), # Випадкові перестановки
    LeaveOneOut()                                             # Leave-one-out
]

# 7. Групи
# Переконайтеся, що 'RAD' має правильну кількість елементів, або створіть масив груп вручну
if 'RAD' in df.columns and len(df['RAD']) == len(X_train):
    groups = df['RAD']
else:
    # Якщо 'RAD' не підходить, створіть власний масив груп
    groups = np.random.randint(0, 5, size=len(X_train))

# 8. Запуск усіх варіантів
for cv in cv_strategies:
    evaluate_model_with_cv('grid', X_train, y_train, cv)
    evaluate_model_with_cv('random', X_train, y_train, cv)

# Використовуємо GroupKFold для групованих даних
evaluate_model_with_cv('grid', X_train, y_train, group_cv.split(X_train, y_train, groups=groups))
evaluate_model_with_cv('random', X_train, y_train, group_cv.split(X_train, y_train, groups=groups))


Grid Search | CV: KFold
Best params: {'C': 2.154434690031882, 'penalty': 'l2', 'solver': 'saga'}
Best score: 0.8698993963782696
------------------------------------------------------------
Random Search | CV: KFold
Best params: {'C': 3.052422429595377, 'penalty': 'l2', 'solver': 'liblinear'}
Best score: 0.8698993963782696
------------------------------------------------------------
Grid Search | CV: KFold
Best params: {'C': 215.44346900318823, 'penalty': 'l1', 'solver': 'liblinear'}
Best score: 0.8614084507042253
------------------------------------------------------------
Random Search | CV: KFold
Best params: {'C': 0.21584494295802448, 'penalty': 'l2', 'solver': 'saga'}
Best score: 0.8642655935613682
------------------------------------------------------------
Grid Search | CV: StratifiedKFold
Best params: {'C': 0.46415888336127775, 'penalty': 'l1', 'solver': 'saga'}
Best score: 0.870020120724346
------------------------------------------------------------
Random Search | CV: Stratif